## 📋 Step 1: Environment Setup

# 🎯 HE-YOLOX-ASFF Training Notebook

### Quick Recovery After Disconnection:
If your Colab session disconnected, run the **Quick Recovery** cell below to resume training instantly!

In [ ]:
# 🚀 QUICK RECOVERY CELL - Run this after disconnection to resume training!
# This single cell does EVERYTHING needed to continue from where you left off

print("🔄 Quick Recovery Mode - Resuming Training...")
print("="*60)

# 1. Mount Google Drive (for dataset and checkpoints)
from google.colab import drive
drive.mount('/content/drive')

# 2. Install dependencies
!pip install -q pyyaml tensorboard thop albumentations pycocotools

# 3. Clone code from GitHub (fresh copy each time)
import os
GITHUB_REPO = "https://github.com/habibour/target_detection_paper.git"

if os.path.exists('/content/code'):
    !rm -rf /content/code
    
print("📥 Cloning code from GitHub...")
!git clone {GITHUB_REPO} /content/code

# Navigate to implement folder
if os.path.exists('/content/code/implement'):
    os.chdir('/content/code/implement')
else:
    os.chdir('/content/code')
print(f"✅ Working directory: {os.getcwd()}")

# 4. Setup paths
PROJECT_DIR = "/content/drive/MyDrive/paper/target_detection"
CHECKPOINT_DIR = f"{PROJECT_DIR}/HE_YOLOX_checkpoints"

# 5. Setup dataset links
!mkdir -p data/VisDrone2019
!ln -sf "{PROJECT_DIR}/VisDrone2019-DET-train" data/VisDrone2019/VisDrone2019-DET-train
!ln -sf "{PROJECT_DIR}/VisDrone2019-DET-val" data/VisDrone2019/VisDrone2019-DET-val
!ln -sf "{PROJECT_DIR}/VisDrone2019-DET-test-dev" data/VisDrone2019/VisDrone2019-DET-test-dev

# 6. Create colab config
import yaml
config_path = 'configs/he_yolox_asff.yaml'
if os.path.exists(config_path):
    with open(config_path, 'r') as f:
        config = yaml.safe_load(f)
    config['data']['data_dir'] = './data/VisDrone2019'
    config['output']['model_dir'] = CHECKPOINT_DIR
    config['output']['log_dir'] = f"{PROJECT_DIR}/HE_YOLOX_logs"
    config['output']['results_dir'] = f"{PROJECT_DIR}/HE_YOLOX_results"
    config['train']['batch_size'] = 16
    config['train']['save_interval'] = 5
    with open('configs/he_yolox_asff_colab.yaml', 'w') as f:
        yaml.dump(config, f)

# 7. Find latest checkpoint and resume
import glob
import re

checkpoints = glob.glob(f"{CHECKPOINT_DIR}/epoch_*.pth")
if checkpoints:
    epoch_nums = []
    for ckpt in checkpoints:
        match = re.search(r'epoch_(\d+)\.pth', ckpt)
        if match:
            epoch_nums.append((int(match.group(1)), ckpt))
    if epoch_nums:
        epoch_nums.sort(key=lambda x: x[0], reverse=True)
        last_epoch, latest_ckpt = epoch_nums[0]
        print(f"\n✅ Found checkpoint at epoch {last_epoch}")
        print(f"   Remaining epochs: {300 - last_epoch}")
        
        print("\n" + "="*60)
        print("🚀 Resuming training...")
        print("="*60)
        !python train.py \
            --config configs/he_yolox_asff_colab.yaml \
            --device cuda \
            --epochs 300 \
            --resume {latest_ckpt}
    else:
        print("\n⚠️ No valid checkpoints found. Run the step-by-step cells for first-time setup.")
else:
    print("\n⚠️ No checkpoint found. This appears to be a fresh start.")
    print("   Run the step-by-step cells below for first-time setup.")

In [ ]:
# Check GPU availability
!nvidia-smi

import torch
print(f"\nPyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

In [ ]:
# Install required packages
!pip install -q pyyaml tensorboard thop albumentations pycocotools

print("✅ Dependencies installed!")

## 💾 Step 2: Mount Google Drive (for checkpoints)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Project paths on Google Drive
PROJECT_DIR = "/content/drive/MyDrive/paper/target_detection"
CHECKPOINT_DIR = f"{PROJECT_DIR}/HE_YOLOX_checkpoints"
LOG_DIR = f"{PROJECT_DIR}/HE_YOLOX_logs"
RESULTS_DIR = f"{PROJECT_DIR}/HE_YOLOX_results"

# Create directories for checkpoints, logs, and results
!mkdir -p "{CHECKPOINT_DIR}"
!mkdir -p "{LOG_DIR}"
!mkdir -p "{RESULTS_DIR}"

# Verify dataset exists
import os
dataset_paths = {
    'train': f"{PROJECT_DIR}/VisDrone2019-DET-train",
    'val': f"{PROJECT_DIR}/VisDrone2019-DET-val",
    'test_dev': f"{PROJECT_DIR}/VisDrone2019-DET-test-dev",
    'test_challenge': f"{PROJECT_DIR}/VisDrone2019-DET-test-challenge"
}

print("📊 Checking dataset availability:")
for split, path in dataset_paths.items():
    if os.path.exists(path):
        images_path = os.path.join(path, 'images')
        if os.path.exists(images_path):
            num_images = len(os.listdir(images_path))
            print(f"  ✅ {split}: {num_images} images found")
        else:
            print(f"  ⚠️ {split}: folder exists but no 'images' subfolder")
    else:
        print(f"  ❌ {split}: NOT FOUND at {path}")

print("\n✅ Google Drive mounted successfully!")

## 📦 Step 3: Clone Code from GitHub

Code is cloned fresh each session to `/content/code` (Colab's local storage).
- **Dataset & Checkpoints**: Stored on Google Drive (persistent)
- **Code**: From GitHub (always latest version)

In [ ]:
# Clone code from GitHub (to /content - fresh each session)
import os

GITHUB_REPO = "https://github.com/habibour/target_detection_paper.git"

# Remove old code if exists and clone fresh
if os.path.exists('/content/code'):
    print("🗑️ Removing old code...")
    !rm -rf /content/code

print("📥 Cloning code from GitHub...")
!git clone {GITHUB_REPO} /content/code

# Navigate to the implement folder
if os.path.exists('/content/code/implement'):
    os.chdir('/content/code/implement')
    print(f"\n📁 Working directory: {os.getcwd()}")
else:
    os.chdir('/content/code')
    print(f"\n📁 Working directory: {os.getcwd()}")

!ls -la
print("\n✅ Code is ready!")

In [ ]:
# Option B: Upload ZIP file (if not using GitHub)
# Uncomment and run this cell only if you prefer manual upload

# from google.colab import files
# import zipfile

# print("📤 Upload your project ZIP file...")
# uploaded = files.upload()

# # Extract the ZIP to /content
# for filename in uploaded.keys():
#     print(f"Extracting {filename}...")
#     with zipfile.ZipFile(filename, 'r') as zip_ref:
#         zip_ref.extractall('/content')
#     print(f"✅ Extracted {filename}")

# # Navigate to project directory
# %cd /content/implement
# !ls -la

## 📊 Step 4: Setup Dataset Paths (Using Existing Data on Drive)

In [ ]:
import os

# Dataset is already on Google Drive - no need to download!
# Just create symbolic links for easier access

PROJECT_DIR = "/content/drive/MyDrive/paper/target_detection"

# Create data directory structure
!mkdir -p data/VisDrone2019

# Create symbolic links to the datasets on Google Drive
!ln -sf "{PROJECT_DIR}/VisDrone2019-DET-train" data/VisDrone2019/VisDrone2019-DET-train
!ln -sf "{PROJECT_DIR}/VisDrone2019-DET-val" data/VisDrone2019/VisDrone2019-DET-val
!ln -sf "{PROJECT_DIR}/VisDrone2019-DET-test-dev" data/VisDrone2019/VisDrone2019-DET-test-dev
!ln -sf "{PROJECT_DIR}/VisDrone2019-DET-test-challenge" data/VisDrone2019/VisDrone2019-DET-test-challenge

# Verify symbolic links
print("\n📊 Dataset Statistics:")
for split in ['train', 'val', 'test-dev', 'test-challenge']:
    images_path = f"data/VisDrone2019/VisDrone2019-DET-{split}/images"
    if os.path.exists(images_path):
        num_images = len(os.listdir(images_path))
        print(f"  {split}: {num_images} images")
    else:
        print(f"  {split}: Not found")

print("\n✅ Dataset paths configured!")

## ⚙️ Step 5: Update Configuration for Colab

In [ ]:
import yaml
import os

PROJECT_DIR = "/content/drive/MyDrive/paper/target_detection"
CHECKPOINT_DIR = f"{PROJECT_DIR}/HE_YOLOX_checkpoints"
LOG_DIR = f"{PROJECT_DIR}/HE_YOLOX_logs"
RESULTS_DIR = f"{PROJECT_DIR}/HE_YOLOX_results"

# Load config
with open('configs/he_yolox_asff.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Update paths for Colab with Google Drive
config['data']['data_dir'] = './data/VisDrone2019'
config['output']['model_dir'] = CHECKPOINT_DIR
config['output']['log_dir'] = LOG_DIR
config['output']['results_dir'] = RESULTS_DIR

# Optimize for Colab (adjust batch size based on GPU memory)
# T4 GPU has 16GB, can handle batch size 16 for 640x640 images
config['train']['batch_size'] = 16
config['val']['batch_size'] = 16
config['train']['num_workers'] = 2  # Colab has limited CPU cores
config['val']['num_workers'] = 2

# Enable checkpoint saving every N epochs for recovery
config['train']['save_interval'] = 5  # Save every 5 epochs

# Save updated config
with open('configs/he_yolox_asff_colab.yaml', 'w') as f:
    yaml.dump(config, f)

print("✅ Configuration updated for Colab!")
print(f"\nSettings:")
print(f"  Batch size: {config['train']['batch_size']}")
print(f"  Checkpoints: {CHECKPOINT_DIR}")
print(f"  Logs: {LOG_DIR}")
print(f"  Results: {RESULTS_DIR}")
print(f"  Save interval: Every 5 epochs")

## 🧪 Step 6: Test Model Architecture

In [ ]:
# Test model can be built
import torch
from models import build_he_yolox

print("Building HE-YOLOX-S model...")
model = build_he_yolox("s", num_classes=13)
model = model.cuda()

# Test forward pass
x = torch.randn(2, 3, 640, 640).cuda()
model.eval()
with torch.no_grad():
    outputs = model(x)

print(f"\n✅ Model test passed!")
print(f"Output shape: {outputs.shape}")

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\nTotal Parameters: {total_params / 1e6:.2f}M")
print(f"Trainable Parameters: {trainable_params / 1e6:.2f}M")

del model, x
torch.cuda.empty_cache()

## 🚀 Step 7: Start Training (with Auto-Resume)

**Important Notes:**
- Training will take 12-18 hours on T4 GPU
- **Checkpoints are saved every 5 epochs to Google Drive** 
- **If session disconnects, just re-run this cell - it will automatically resume from the last checkpoint!**
- Best model is automatically saved as `best.pth` when validation improves
- All weights are safely stored on Google Drive

In [ ]:
# ⚡ ANTI-DISCONNECT: Run this to prevent Colab from disconnecting due to inactivity
# This creates a JavaScript snippet that clicks to keep the session alive

from IPython.display import display, Javascript
import time

def keep_alive():
    """Display JavaScript to prevent Colab timeout"""
    display(Javascript('''
        function ClickConnect(){
            console.log("Keeping Colab alive...");
            document.querySelector("colab-connect-button").click()
        }
        setInterval(ClickConnect, 60000)
    '''))
    print("✅ Anti-disconnect enabled! Session will stay alive.")

# Uncomment the line below to enable (may not work in all cases)
# keep_alive()

print("💡 Tip: To prevent disconnection, you can also:")
print("   1. Keep the browser tab active")
print("   2. Use Colab Pro for longer runtime")
print("   3. Run a simple loop in browser console:")
print("      setInterval(() => { console.log('alive'); }, 60000)")

In [ ]:
import os
import glob
import re

PROJECT_DIR = "/content/drive/MyDrive/paper/target_detection"
CHECKPOINT_DIR = f"{PROJECT_DIR}/HE_YOLOX_checkpoints"

def find_latest_checkpoint():
    """Find the latest checkpoint to resume from"""
    # Look for epoch checkpoints
    epoch_checkpoints = glob.glob(f"{CHECKPOINT_DIR}/epoch_*.pth")
    
    if not epoch_checkpoints:
        return None, 0
    
    # Extract epoch numbers and find the latest
    epoch_nums = []
    for ckpt in epoch_checkpoints:
        match = re.search(r'epoch_(\d+)\.pth', ckpt)
        if match:
            epoch_nums.append((int(match.group(1)), ckpt))
    
    if epoch_nums:
        epoch_nums.sort(key=lambda x: x[0], reverse=True)
        return epoch_nums[0][1], epoch_nums[0][0]
    
    return None, 0

# Check for existing checkpoint
latest_checkpoint, last_epoch = find_latest_checkpoint()
best_checkpoint = f"{CHECKPOINT_DIR}/best.pth"

print("="*60)
print("🔍 Checkpoint Status:")
print("="*60)

if latest_checkpoint:
    print(f"✅ Found checkpoint: {os.path.basename(latest_checkpoint)}")
    print(f"   Last completed epoch: {last_epoch}")
    resume_flag = f"--resume {latest_checkpoint}"
    remaining_epochs = 300 - last_epoch
    print(f"   Remaining epochs: {remaining_epochs}")
else:
    print("📝 No checkpoint found. Starting fresh training.")
    resume_flag = ""
    remaining_epochs = 300

if os.path.exists(best_checkpoint):
    print(f"⭐ Best model exists at: {os.path.basename(best_checkpoint)}")

print("="*60)
print(f"\n🚀 Starting training...")
print(f"   Config: configs/he_yolox_asff_colab.yaml")
print(f"   Total epochs: 300")
if resume_flag:
    print(f"   Resuming from epoch: {last_epoch + 1}")
print("="*60)

# Start training with auto-resume
!python train.py \
    --config configs/he_yolox_asff_colab.yaml \
    --device cuda \
    --epochs 300 \
    {resume_flag}

## 📊 Step 8: Monitor Training (Optional)

Run this in a separate cell while training to monitor progress

In [ ]:
# Load TensorBoard
%load_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/HE_YOLOX/logs

In [ ]:
# Monitor GPU usage
!nvidia-smi

## 📈 Step 9: Evaluate Model

In [ ]:
import os

PROJECT_DIR = "/content/drive/MyDrive/paper/target_detection"
CHECKPOINT_DIR = f"{PROJECT_DIR}/HE_YOLOX_checkpoints"
RESULTS_DIR = f"{PROJECT_DIR}/HE_YOLOX_results"

# Check which weights to use
best_weights = f"{CHECKPOINT_DIR}/best.pth"
if os.path.exists(best_weights):
    weights_path = best_weights
    print(f"✅ Using best model: {weights_path}")
else:
    # Find latest epoch checkpoint
    import glob
    checkpoints = sorted(glob.glob(f"{CHECKPOINT_DIR}/epoch_*.pth"))
    if checkpoints:
        weights_path = checkpoints[-1]
        print(f"⚠️ Best model not found, using latest checkpoint: {weights_path}")
    else:
        print("❌ No weights found! Please train the model first.")
        weights_path = None

if weights_path:
    # Evaluate on validation set
    !python eval.py \
        --config configs/he_yolox_asff_colab.yaml \
        --weights {weights_path} \
        --split val \
        --device cuda

    # Display results
    results_file = f"{RESULTS_DIR}/eval_results.txt"
    print("\n" + "="*60)
    print("📊 Evaluation Results:")
    print("="*60)
    if os.path.exists(results_file):
        with open(results_file, 'r') as f:
            print(f.read())
    else:
        print("Results file not found. Check logs for evaluation output.")

## 🎯 Step 10: Test Inference

In [ ]:
import glob
import random
import shutil
import os

PROJECT_DIR = "/content/drive/MyDrive/paper/target_detection"
CHECKPOINT_DIR = f"{PROJECT_DIR}/HE_YOLOX_checkpoints"
RESULTS_DIR = f"{PROJECT_DIR}/HE_YOLOX_results"

# Create test directory
!mkdir -p test_images

# Copy 5 random validation images
val_images = glob.glob('data/VisDrone2019/VisDrone2019-DET-val/images/*.jpg')
if val_images:
    selected_images = random.sample(val_images, min(5, len(val_images)))
    for img in selected_images:
        shutil.copy(img, 'test_images/')
    print(f"✅ Selected {len(selected_images)} test images")
else:
    print("❌ No validation images found!")
    selected_images = []

# Find weights
best_weights = f"{CHECKPOINT_DIR}/best.pth"
if os.path.exists(best_weights):
    weights_path = best_weights
else:
    checkpoints = sorted(glob.glob(f"{CHECKPOINT_DIR}/epoch_*.pth"))
    weights_path = checkpoints[-1] if checkpoints else None

if weights_path and selected_images:
    # Run inference
    !python inference.py \
        --config configs/he_yolox_asff_colab.yaml \
        --weights {weights_path} \
        --source test_images/ \
        --output {RESULTS_DIR}/inference \
        --conf_thresh 0.3 \
        --save_img \
        --device cuda

    print(f"\n✅ Inference complete! Results saved to: {RESULTS_DIR}/inference")
else:
    if not weights_path:
        print("❌ No trained weights found!")
    if not selected_images:
        print("❌ No test images available!")

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import glob
import os

PROJECT_DIR = "/content/drive/MyDrive/paper/target_detection"
RESULTS_DIR = f"{PROJECT_DIR}/HE_YOLOX_results"

# Find result images
result_images = glob.glob(f'{RESULTS_DIR}/inference/*.jpg')
result_images.extend(glob.glob(f'{RESULTS_DIR}/inference/*.png'))

if result_images:
    # Display up to 4 results in a 2x2 grid
    num_display = min(4, len(result_images))
    rows = (num_display + 1) // 2
    
    fig, axes = plt.subplots(rows, 2, figsize=(16, 8 * rows))
    axes = axes.flatten() if num_display > 1 else [axes]
    
    for idx, img_path in enumerate(result_images[:num_display]):
        img = Image.open(img_path)
        axes[idx].imshow(img)
        axes[idx].axis('off')
        axes[idx].set_title(f'Detection Result {idx+1}', fontsize=12)
    
    # Hide empty subplots
    for idx in range(num_display, len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n📊 Total inference results: {len(result_images)}")
else:
    print("❌ No inference results found. Run inference first!")

## 💾 Step 11: Download Results

In [ ]:
import shutil
import os
from google.colab import files

PROJECT_DIR = "/content/drive/MyDrive/paper/target_detection"
CHECKPOINT_DIR = f"{PROJECT_DIR}/HE_YOLOX_checkpoints"
RESULTS_DIR = f"{PROJECT_DIR}/HE_YOLOX_results"

print("📦 Creating results archive...")

# Create a temporary directory with key files
!mkdir -p /content/export_results
!cp -r {RESULTS_DIR}/* /content/export_results/ 2>/dev/null || echo "No results yet"

# Copy best model if exists
if os.path.exists(f"{CHECKPOINT_DIR}/best.pth"):
    !cp {CHECKPOINT_DIR}/best.pth /content/export_results/

# Create archive
shutil.make_archive('/content/he_yolox_results', 'zip', '/content/export_results')

print("\n📥 Download starting...")
files.download('/content/he_yolox_results.zip')

print("\n✅ Results downloaded! Contains:")
print("  - Best model weights (best.pth)")  
print("  - Evaluation results")
print("  - Inference visualizations")
print("\n💡 Note: All checkpoints remain safely on Google Drive at:")
print(f"   {CHECKPOINT_DIR}")

## 📊 Training Summary

### Expected Results (from paper):

| Class | Target AP (%) |
|-------|---------------|
| Car | 81.2 |
| Bus | 66.4 |
| Truck | 55.6 |
| Pedestrian | 42.6 |
| Motor | 45.5 |
| Bicycle | 19.4 |

### Files on Google Drive:
```
/content/drive/MyDrive/paper/target_detection/
├── VisDrone2019-DET-train/          # Training data
├── VisDrone2019-DET-val/            # Validation data  
├── VisDrone2019-DET-test-dev/       # Test data
├── VisDrone2019-DET-test-challenge/ # Challenge test data
├── HE_YOLOX_checkpoints/            # Model weights
│   ├── best.pth                     # Best model
│   └── epoch_*.pth                  # Epoch checkpoints
├── HE_YOLOX_logs/                   # TensorBoard logs
└── HE_YOLOX_results/                # Evaluation & inference
```

### 🔄 Recovery After Disconnection:
1. Re-run **Cell 1-4** (GPU check, install deps, mount drive, setup paths)
2. Skip to **Cell 7** (Training) - it will automatically resume from last checkpoint!

### Tips:
1. **Checkpoints saved every 5 epochs** to Google Drive
2. **Auto-resume:** Just re-run training cell after disconnection  
3. **Best model:** Automatically saved when validation improves
4. **Full training:** Takes ~12-18 hours on T4 GPU

---

**🎉 Congratulations! You've successfully set up HE-YOLOX-ASFF training!**

## 🚀 Next Steps

1. **Compare with paper results** - Check if your mAP matches the paper
2. **Fine-tune hyperparameters** - Adjust learning rate, batch size
3. **Test on custom images** - Upload your own drone images
4. **Export model** - Convert to ONNX for deployment
5. **Experiment** - Try different model sizes (M, L, X)

### Resources:
- Paper: DOI 10.1109/ICDSCNC62492.2024.10939462
- VisDrone Dataset: http://aiskyeye.com/
- Your results: `/content/drive/MyDrive/HE_YOLOX/`